In [15]:
import cv2
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

import mediapipe as mp

In [16]:
MODEL_PATH = '../data/models/pose_landmarker_heavy.task'
VIDEO_INPUT_PATH = "../datasets/videos/merged.mp4"
VIDEO_OUTPUT_PATH = "../data/results/pose_output.mp4"
CSV_OUTPUT_PATH = "../data/results/pose_metrics.csv"
MAX_GAP = 2
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21),
    (17, 19), (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20),
    (11, 23), (12, 24), (23, 24), (23, 25), (24, 26), (25, 27), (26, 28),
    (27, 29), (28, 30), (29, 31), (30, 32), (27, 31), (28, 32)
]
TORSO_JOINTS = [11, 12, 23, 24]  

In [17]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


In [ ]:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=VisionRunningMode.VIDEO,
    output_segmentation_masks=False,
    min_pose_presence_confidence=0.4,
    min_tracking_confidence=0.4
)

def draw_landmarks_from_landmarks(rgb_image, pose_landmarks):
    annotated_image = rgb_image.copy()
    h, w, _ = annotated_image.shape

    pts = []
    for lm in pose_landmarks:
        x = int(lm.x * w)
        y = int(lm.y * h)
        pts.append((x, y))

    for s, e in POSE_CONNECTIONS:
        if s < len(pts) and e < len(pts):
            cv2.line(annotated_image, pts[s], pts[e], (0, 255, 0), 2)

    for lm in pose_landmarks:
        if lm.visibility < 0.5:
            continue
        x = int(lm.x * w)
        y = int(lm.y * h)
        cv2.circle(annotated_image, (x, y), 5, (0, 0, 255), -1)

    return annotated_image

last_valid_landmarks = None
missing_count = 0
pose_used = None
interpolated = False

with PoseLandmarker.create_from_options(options) as landmarker:
    cap = cv2.VideoCapture(VIDEO_INPUT_PATH)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {VIDEO_INPUT_PATH}")
        exit()
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {frame_width}x{frame_height} @ {fps} FPS, {total_frames} frames")
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(VIDEO_OUTPUT_PATH, fourcc, fps, (frame_width, frame_height))
    
    pose_data = []
    
    frame_count = 0
    
    print("Processing video frames...")
    with tqdm(total=total_frames, desc="Processing") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                break
            
            frame_timestamp_ms = int((frame_count / fps) * 1000)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
            pose_landmarker_result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)
            
            if pose_landmarker_result.pose_landmarks:
                last_valid_landmarks = pose_landmarker_result.pose_landmarks[0]
                missing_count = 0
                pose_used = last_valid_landmarks
                interpolated = False
            else:
                missing_count += 1
                if last_valid_landmarks is not None and missing_count <= MAX_GAP:
                    stable = all(
                        lm.visibility > 0.7
                        for i, lm in enumerate(last_valid_landmarks)
                        if i in TORSO_JOINTS
                    )

                    if stable:
                        pose_used = last_valid_landmarks
                        interpolated = True
                    else:
                        pose_used = None
                        interpolated = False
                else:
                    pose_used = None
                    interpolated = False
                    
            if pose_used is not None:
                annotated_image = draw_landmarks_from_landmarks(rgb_frame, pose_used)
            else:
                annotated_image = rgb_frame
            annotated_bgr = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
            
            out.write(annotated_bgr)
            
            frame_data = {
                "frame": frame_count,
                "timestamp_ms": frame_timestamp_ms,
                "model_pose_detected": pose_landmarker_result.pose_landmarks is not None,
                "pose_used": pose_used is not None,
                "interpolated": interpolated,
                "missing_streak": missing_count
            }

            if pose_used is not None:
                for i, lm in enumerate(pose_used):
                    frame_data[f"lm_{i}_x"] = lm.x
                    frame_data[f"lm_{i}_y"] = lm.y
                    frame_data[f"lm_{i}_z"] = lm.z
                    frame_data[f"lm_{i}_vis"] = lm.visibility
            else:
                for i in range(33):
                    frame_data[f"lm_{i}_x"] = None
                    frame_data[f"lm_{i}_y"] = None
                    frame_data[f"lm_{i}_z"] = None
                    frame_data[f"lm_{i}_vis"] = None

            pose_data.append(frame_data)
            frame_count += 1
            pbar.update(1)
    
    cap.release()
    out.release()
    
    if pose_data:
        df = pd.DataFrame(pose_data)
        df.to_csv(CSV_OUTPUT_PATH, index=False)
        print(f"\nPose metrics saved to: {CSV_OUTPUT_PATH}")
        print(f"Total records: {len(df)}")
    else:
        print("\nNo pose landmarks detected in the video.")
    
    print(f"Annotated video saved to: {VIDEO_OUTPUT_PATH}")
    print("Processing complete!")





IndentationError: expected an indented block after 'else' statement on line 119 (1847533718.py, line 121)